In [ ]:
import datetime

import numpy as np
import polars as pl
import matplotlib.pyplot as plt
from tqdm import tqdm

import stock

In [ ]:
def plot_with_rs(df, start_date, end_date=datetime.datetime.today()):
    fig = plt.figure()
    df = df.filter((pl.col("date") >= start_date) & (pl.col("date") <= end_date))
    ax1 = fig.add_subplot(111)
    ax1.plot(df["date"], df["close"], "red", label="close")

    ax2 = ax1.twinx()
    ax2.plot(df["date"], df["rs"], "blue", label="rs")
    ax2.grid("both")
    ax2.tick_params(axis='x', labelrotation=90)

    plt.legend()

In [ ]:
def read_csv(csv_path):
    df = pl.read_csv(csv_path)
    columns = [
        pl.col("date").str.to_datetime("%Y/%m/%d").cast(pl.Date),
        pl.col("open").cast(pl.Float64),
        pl.col("high").cast(pl.Float64),
        pl.col("low").cast(pl.Float64),
        pl.col("close").cast(pl.Float64),
        pl.col("volume").cast(pl.Int64),
        pl.col("rs_nikkei").cast(pl.Float64),
        pl.col("rs_topix").cast(pl.Float64),
    ]
    df = df.select(columns)
    return df

csv_dir = stock.PROJECT_ROOT / "data" / "daily"

In [ ]:
code ="4463"
date = datetime.datetime(year=2024, month=5, day=1).date()
start_date = date - datetime.timedelta(days=365)

csv_path = csv_dir / f"{code}.csv"
df = stock.kabutan.read_data_csv(csv_path)
ref_df = stock.kabutan.read_data_csv(csv_dir / "0010.csv", exclude_none=False)   # topix
#rs = relative_strength(df, ref_df, start_date=start_date, end_date=date)

In [ ]:
def calc_rs(dates, days=30):
    min_date = df["date"].min()
    res = -np.ones(len(dates))
    for idx, date in enumerate(sorted(dates)):
        start_date = date - datetime.timedelta(days=days)
        if start_date < min_date:
            continue
        res[idx] = stock.relative_strength.relative_strength_v2(df, ref_df, start_date=start_date, end_date=date)
    return pl.Series(res)

In [ ]:
ref_df = stock.kabutan.read_data_csv(csv_dir / "0010.csv", exclude_none=False)   # topix
for csv_path in tqdm(sorted(csv_dir.glob("*.csv"))):
    df = stock.kabutan.read_data_csv(csv_path, exclude_none=True)
    df = df.with_columns(pl.col("date").map_batches(calc_rs).alias("rs"))
    stock.kabutan.write_data_csv(df, csv_path)

In [ ]:
csv_path = csv_dir / "0000.csv"
df = read_csv(csv_path)   # topix
df = df.with_columns(pl.col("date").map_batches(calc_rs).alias("rs"))
ref_df = df
stock.kabutan.write_data_csv(df, csv_path)